1. Load the package

In [ ]:
import scipy.io
import random
import numpy as np
import torch
import torch.nn as nn
import os
import torchvision
import time
from torchvision import models
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from torch.autograd.function import Function
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim.lr_scheduler as lr_scheduler

Load Office-Caltech-10 dataset


You need to download [Office-Caltech-10 DeCAF-fc6 features](https://drive.google.com/drive/folders/1XAKj5ikc6ygcanexeiWuwvnHMf5rUVGy) generated in [1] to run this code. Then: 


2.1. If you run the code on Colab, you will need to put the data in the corresponding folder of your [Google Drive](https://drive.google.com/drive/u/0/my-drive).


2.2. If you run the code locally, you will need to put the data in the corresponding folder of your device.



[1] Donahue, J., Jia, Y., Vinyals, O., Hoffman, J., Zhang, N., Tzeng, E., & Darrell, T. (2014, January). Decaf: A deep convolutional activation feature for generic visual recognition. In International conference on machine learning (pp. 647-655). PMLR.

In [ ]:
# Mount the Google drive, please ignore this cell if you run the code locally.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
amazon = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/domain_adaptation_master/office-31/fc6/amazon_fc6.mat')
webcam = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/domain_adaptation_master/office-31/fc6/webcam_fc6.mat')
dslr = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/domain_adaptation_master/office-31/fc6/dslr_fc6.mat')
cal  = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/domain_adaptation_master/office-31/fc6/caltech_decaf.mat')

In [ ]:
amazon_fts,amazon_labels=amazon['fts'],amazon['labels'].reshape(-1)-1
webcam_fts,webcam_labels=webcam['fts'],webcam['labels'].reshape(-1)-1
dslr_fts,dslr_labels=dslr['fts'],dslr['labels'].reshape(-1)-1
cal_fts,cal_labels=cal['feas'],cal['labels'].reshape(-1)-1

3. Define functions to formulate the training and testing sets

In [ ]:
np.random.choice
class_set=np.array([1,2,6,11,12,13,16,17,18,23])-1
#backpack, bike, calculator, headphones, keyboard, laptop-computer, monitor, mouse, mu, and projector

def get_data(domain_name):
  index=np.array([])
  label_for_training=[]
  fts,labels=None, None
  if domain_name=='amazon':
    fts,labels=amazon_fts,amazon_labels
  elif domain_name=='webcam':
    fts,labels=webcam_fts,webcam_labels
  elif domain_name=='dslr':
    fts,labels=dslr_fts,dslr_labels

  j=0
  for i in class_set:
    index=np.hstack((index,np.where(labels==i)[0]))
    for a in np.where(labels==i)[0]:
      label_for_training.append(j)
    j=j+1
  index=index.astype(int)
  return fts[index],np.array(label_for_training)

def get_index(feature,labels,select_num):
  select_list=np.array([])
  remaining_list=np.array([])
  for i in range(10):
    full_list=np.where(labels==i)[0]
    ran_select=np.random.choice(full_list,select_num,replace=False)
    select_list=np.hstack((select_list,ran_select))


    remain_list=np.setdiff1d(full_list,ran_select)
    remaining_list=np.hstack((remaining_list,remain_list))
  return select_list.astype(int),remaining_list.astype(int)

  


def souce_target_split(src_name, tar_name):
  if src_name =='cad':
    src_fea,src_label=np.array(cal_fts),np.array(cal_labels)
  else:
    src_fea,src_label=get_data(src_name)
  if tar_name == 'cad':
    tar_fea,tar_label=np.array(cal_fts),np.array(cal_labels)
  else:
    tar_fea,tar_label=get_data(tar_name)

  if src_name=='amazon':
    scr_number=20
  elif src_name=='cad':
    scr_number=20
  else:
    scr_number=8
  tar_number=3
  
  src_select,src_remain=get_index(src_fea,src_label,scr_number)
  tar_select,tar_remain=get_index(tar_fea,tar_label,tar_number)
  

  return src_fea[src_select],src_label[src_select],tar_fea[tar_select],tar_label[tar_select],tar_fea[tar_remain],tar_label[tar_remain]

4. Generate training pairs

In [ ]:
# Initialization.Create_Pairs
class TrainSet(Dataset):
    def __init__(self, src_name, tar_name):
        self.train_x_scr,self.train_y_scr,self.train_x_tar,self.train_y_tar,self.test_x_tar,self.test_y_tar = souce_target_split(src_name, tar_name)
                
        Training_P=[]
        Training_N=[]
        for trs in range(len(self.train_y_scr)):
            for trt in range(len(self.train_y_tar)):
                if self.train_y_scr[trs] == self.train_y_tar[trt]:
                    Training_P.append([trs,trt, 1])
                else:
                    Training_N.append([trs,trt, 0])
        print("Class P : ", len(Training_P), " N : ", len(Training_N))
        
        random.shuffle(Training_N)
        self.imgs = Training_P+Training_N[:3*len(Training_P)]
        random.shuffle(self.imgs)

    def __getitem__(self, idx):
        src_idx, tgt_idx, domain = self.imgs[idx]

        x_src, y_src = self.train_x_scr[src_idx], self.train_y_scr[src_idx]
        x_tgt, y_tgt = self.train_x_tar[tgt_idx], self.train_y_tar[tgt_idx]

        x_src = torch.from_numpy(x_src)
        x_tgt = torch.from_numpy(x_tgt)

        return x_src, y_src, x_tgt, y_tgt

    def __len__(self):
        return len(self.imgs)

class TestSet(Dataset):
    def __init__(self, src_name, tar_name):
        self.train_x_scr,self.train_y_scr,self.train_x_tar,self.train_y_tar,self.test_x_tar,self.test_y_tar = souce_target_split(src_name, tar_name)

    def __getitem__(self, idx):
        x, y = self.test_x_tar[idx], self.test_y_tar[idx]
        x = torch.from_numpy(x)
        return x, y

    def __len__(self):
        return len(self.test_x_tar)

5. Define the network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(4096, 1024)
        self.act1= nn.PReLU()
        self.layer2 = nn.Linear(1024, 128)
        self.act2= nn.PReLU()
        self.ip2 = nn.Linear(128, 10)
  

    def forward(self, x):
        x = x.view(-1, 4096)
        x = self.layer1(x)
        x = self.act1(x)
        feature = self.layer2(x)
        feature = self.act2(feature)
        ip2 = self.ip2(feature)
        return feature , ip2

    def predict(self, x):
        return self.forward(x)

6. Define CCSA loss

In [ ]:
# Constrastive Semantic Alignment Loss
def csa_loss(x, y, class_eq):
    margin = 1
    dist = F.pairwise_distance(x, y)
    loss = class_eq * dist.pow(2)
    loss += (1 - class_eq) * (margin - dist).clamp(min=0).pow(2)
    return loss.mean()

7. Define the function for training and testing 

In [ ]:
def trainandtest(model, dataloaders, optimizer,loss_weight):
    n_epoch = 100
    criterion = nn.CrossEntropyLoss()
    since = time.time()
    best_acc = 0
    stop = 0
    max_acc=[]
    for epoch in range(0, n_epoch):
        stop += 1

        for phase in ['src']:
            if phase == 'src':
                model.train()
            else:
                model.eval()
            total_loss, total_center,correct = 0, 0,0
            for src_img, src_label, target_img, target_label in dataloaders[phase]:
                src_img, src_label, target_img, target_label = src_img.cuda(), src_label.cuda(), target_img.cuda(), target_label.cuda()
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'src'):
                    src_img = src_img.to(torch.float32)
                    target_img = target_img.to(torch.float32)
                    src_feat, src_pre = model(src_img)
                    tgt_feat, tgt_pre = model(target_img)
                    softmax=criterion(src_pre, src_label)
                    csa = csa_loss(src_feat, tgt_feat,
                       (src_label == target_label).float())

                    loss = (1-loss_weight) * softmax + loss_weight * csa
                if phase == 'src':
                    loss.backward()
                    optimizer.step()
        model.eval()
        correct = 0
        for inputs, labels in dataloaders['tar']:
          inputs, labels= inputs.cuda(),labels.cuda()
          inputs = inputs.to(torch.float32)
          ip1, outputs = model(inputs)
          preds = torch.max(outputs, 1)[1]
          correct += torch.sum(preds == labels.data)
        epoch_acc = correct.double() / len(dataloaders['tar'].dataset)
        if epoch>97:
          print('Testing acc:',epoch_acc.item())
        max_acc.append(epoch_acc)
    print(max(max_acc))




8. Define evaluation function

In [ ]:
def evaluate(src_name,tar_name,seeds):
  batch_size = 24
  n_class=10

  random.seed(seeds) #amend if you want

  train_set = TrainSet(src_name, tar_name)
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
  test_set = TestSet(src_name, tar_name)
  test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  print("Dataset Length Train : ", len(train_set), " Test : ", len(test_set))

  model = Net().cuda()
  param_group = []
  learning_rate = 0.0001
  momentum = 5e-4
  for k, v in model.named_parameters():


    if k.__contains__('classifier'):
        param_group += [{'params': v, 'lr': learning_rate}]
    elif k.__contains__('ip'):
        param_group += [{'params': v, 'lr': learning_rate}]
    else:
        param_group += [{'params': v, 'lr': learning_rate}]


  
  optimizer = torch.optim.Adam(param_group)
  
  dataloaders = {'src': train_loader,
               'tar': test_loader}
  trainandtest(model, dataloaders, optimizer,0.25)

9. Example

Example below is the *Amaon and DSLR* domain adaptation task.
You may change the value of variables to get other experimental results.

Note that models will be trained from scratch. Training time should last for around **4 minutes** for one repetition if you use the GPU (GeForce RTX 3090).

Although the results you get may be slightly different from the ones of the manuscript due to randomized initialization, the gap should be small.

In [ ]:
#'amazon', 'webcam','dslr','cad'
source_domain='amazon'
target_domain='dslr'
random_seed=1
evaluate(source_domain,target_domain,random_seed)

Class P :  600  N :  5400
Dataset Length Train :  2400  Test :  127
Testing acc: 0.889763779527559
Testing acc: 0.8976377952755905
tensor(0.9055, device='cuda:0', dtype=torch.float64)


In [ ]:
#'amazon', 'webcam','dslr','cad'
source_domain='dslr'
target_domain='amazon'
random_seed=1
evaluate(source_domain,target_domain,random_seed)

Class P :  240  N :  2160
Dataset Length Train :  960  Test :  928
Testing acc: 0.8868534482758621
Testing acc: 0.8793103448275862
tensor(0.8944, device='cuda:0', dtype=torch.float64)
